In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset
dataset = load_dataset("web_questions")


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/260k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/142k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3778 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2032 [00:00<?, ? examples/s]

##---Print rows names

In [ ]:
print(dataset["train"][0])

{'url': 'http://www.freebase.com/view/en/justin_bieber', 'question': 'what is the name of justin bieber brother?', 'answers': ['Jazmyn Bieber', 'Jaxon Bieber']}


##---Print column names

In [ ]:
print(dataset["test"].column_names)

['url', 'question', 'answers']


In [ ]:
print(dataset["train"].column_names)

['url', 'question', 'answers']


Intermediate Dataset: nq_open

In [ ]:
from datasets import load_dataset
dataset = load_dataset("nq_open")


README.md: 0.00B [00:00, ?B/s]

nq_open/train-00000-of-00001.parquet:   0%|          | 0.00/4.46M [00:00<?, ?B/s]

nq_open/validation-00000-of-00001.parque(…):   0%|          | 0.00/214k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87925 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3610 [00:00<?, ? examples/s]

In [ ]:
print(dataset["train"][0])


{'question': 'where did they film hot tub time machine', 'answer': ['Fernie Alpine Resort']}


In [ ]:
print(dataset["train"].column_names)

['question', 'answer']


#Classification Dataset


In [ ]:
from datasets import load_dataset
ds = load_dataset("imdb")



In [ ]:
print(ds["train"],[0])

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
}) [0]


In [ ]:
print(ds["train"],"\n"[0])

Dataset({
    features: ['text', 'label'],
    num_rows: 25000
}) 



In [ ]:
from google.colab import userdata
import os

# It's recommended to store your API key as a Colab secret.
# Go to the 'Secrets' tab (🔑 icon on the left panel),
# add a new secret named 'OPENAI_API_KEY' with your actual API key as the value,
# and make sure to enable 'Notebook access'.

# If you prefer to paste it directly (less secure for sharing):
# os.environ["OPENAI_API_KEY"] = "YOUR_ACTUAL_OPENAI_API_KEY_HERE"

# Load the API key from Colab secrets
os.environ["OPENAI_API_KEY"] = userdata.get('Openai_key')

print("OpenAI API key loaded.")

OpenAI API key loaded.


In [ ]:
import os
import random
from datasets import load_dataset
from openai import OpenAI
from tqdm import tqdm

# -----------------------------
# CONFIG
# -----------------------------
DATASET_NAME = "stanfordnlp/imdb"
SAMPLE_SIZE = 500
SEED = 42

MODELS = {
    "gpt-5.2": "gpt-5.2",
    "gpt-5-mini": "gpt-5-mini"
}

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

# -----------------------------
# LOAD DATASET
# -----------------------------
print(f"\n--- Benchmarking: {DATASET_NAME} ---")
dataset = load_dataset(DATASET_NAME)

split = "train" if "train" in dataset else list(dataset.keys())[0]
data = list(dataset[split])

random.seed(SEED)
samples = random.sample(data, min(SAMPLE_SIZE, len(data)))

# -----------------------------
# AUTO-DETECT COLUMNS
# -----------------------------
sample = samples[0]

TEXT_COLUMN = "text"     # IMDb is fixed & known
LABEL_COLUMN = "label"   # IMDb is fixed & known

print(f"Detected TEXT_COLUMN: '{TEXT_COLUMN}', LABEL_COLUMN: '{LABEL_COLUMN}'")
print(f"Using {len(samples)} samples from '{split}' split")

# -----------------------------
# IMDb-SPECIFIC LABEL MEANING
# -----------------------------
LABEL_MEANING = {
    0: "negative",
    1: "positive"
}

# -----------------------------
# SYSTEM PROMPT (IMDb-CORRECT)
# -----------------------------
SYSTEM_PROMPT = """
You are a sentiment classification system.

Classify the sentiment of the given movie review.

Label meanings:
- 0 = negative sentiment
- 1 = positive sentiment

Rules:
- Return ONLY the number 0 or 1
- Do NOT return words
- Do NOT explain
"""

# -----------------------------
# RUN MODEL
# -----------------------------
def run_model(model_id, text):
    response = client.chat.completions.create(
        model=model_id,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": text}
        ]
    )
    return response.choices[0].message.content.strip()

# -----------------------------
# EVALUATION
# -----------------------------
results = {name: 0 for name in MODELS}

for item in tqdm(samples, desc="Evaluating"):
    text = item[TEXT_COLUMN]
    true_label = str(item[LABEL_COLUMN])  # "0" or "1"

    for name, model_id in MODELS.items():
        pred = run_model(model_id, text)

        if pred == true_label:
            results[name] += 1

# -----------------------------
# RESULTS
# -----------------------------
print("\n==============================")
print(f"📊 RESULTS — {DATASET_NAME}")
print("==============================")

for name in MODELS:
    acc = (results[name] / len(samples)) * 100
    print(f"{name} accuracy: {acc:.2f}%")



--- Benchmarking: stanfordnlp/imdb ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

plain_text/test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

...

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Detected TEXT_COLUMN: 'text', LABEL_COLUMN: 'label'
Using 500 samples from 'train' split


Evaluating: 100%|██████████| 500/500 [20:06<00:00,  2.41s/it]


📊 RESULTS — stanfordnlp/imdb
gpt-5.2 accuracy: 94.60%
gpt-5-mini accuracy: 93.80%


In [ ]:
DATASETS_TO_BENCHMARK = [
    "stanfordnlp/imdb",
    "ag_news",
    "yelp_polarity",
    "dbpedia_14",
    "dair-ai/emotion",
    "CogComp/trec",
    "SetFit/amazon_polarity",
    "glue/sst2"
]

MODELS = {
    "gpt-5.2": "gpt-5.2",
    "gpt-5-mini": "gpt-5-mini"
}

print("Datasets and models defined.")

In [ ]:
import os
import random
import asyncio
from datasets import load_dataset, ClassLabel
from openai import OpenAI
from tqdm.asyncio import tqdm

# Assuming OpenAI client and MODELS are defined globally as in previous steps
# If they were not, they would need to be re-initialized or passed as arguments.
# For this refactoring, we'll assume the global client and MODELS are available.

# client = OpenAI(api_key=os.environ["OPENAI_API_KEY"]) # This was in a previous cell
# MODELS = { "gpt-5.2": "gpt-5.2", "gpt-5-mini": "gpt-5-mini" } # This was in a previous cell

MAX_CONCURRENT_REQUESTS = 5 # Define the maximum concurrent requests

def benchmark_dataset(dataset_name, sample_size, seed):
    print(f"\n--- Benchmarking: {dataset_name} ---")

    # -----------------------------
    # LOAD DATASET
    # -----------------------------
    try:
        dataset = load_dataset(dataset_name)
    except Exception as e:
        print(f"Error loading dataset {dataset_name}: {e}")
        return {name: 0.0 for name in MODELS} # Return 0 accuracy for failed dataset

    split = "train" if "train" in dataset else list(dataset.keys())[0]
    data = list(dataset[split])

    random.seed(seed)
    samples = random.sample(data, min(sample_size, len(data)))

    # -----------------------------
    # DYNAMIC COLUMN DETECTION
    # -----------------------------
    TEXT_COLUMN = None
    LABEL_COLUMN = None

    # Common text column names
    text_col_candidates = ["text", "sentence", "review", "question", "tweet", "comment"]
    # Common label column names
    label_col_candidates = ["label", "labels", "target", "sentiment", "class"]

    features = dataset[split].features
    column_names = dataset[split].column_names

    for col in text_col_candidates:
        if col in column_names:
            TEXT_COLUMN = col
            break

    for col in label_col_candidates:
        if col in column_names:
            # Check if it's a ClassLabel type for better confidence
            if isinstance(features[col], ClassLabel):
                LABEL_COLUMN = col
                break
    # If not found by name, check by type for ClassLabel
    if not LABEL_COLUMN:
        for col_name, feature_type in features.items():
            if isinstance(feature_type, ClassLabel):
                LABEL_COLUMN = col_name
                break

    if not TEXT_COLUMN:
        print(f"Warning: Could not dynamically detect TEXT_COLUMN for {dataset_name}. Skipping dataset.")
        return {name: 0.0 for name in MODELS}
    if not LABEL_COLUMN:
        print(f"Warning: Could not dynamically detect LABEL_COLUMN for {dataset_name}. Skipping dataset.")
        return {name: 0.0 for name in MODELS}

    print(f"Detected TEXT_COLUMN: '{TEXT_COLUMN}', LABEL_COLUMN: '{LABEL_COLUMN}'")
    print(f"Using {len(samples)} samples from '{split}' split")

    # -----------------------------
    # LABEL MEANING (Dynamically create based on ClassLabel feature)
    # -----------------------------
    LABEL_MEANING = {}
    if isinstance(features[LABEL_COLUMN], ClassLabel):
        for idx, name in enumerate(features[LABEL_COLUMN].names):
            LABEL_MEANING[idx] = name
    else:
        print(f"Warning: Label column '{LABEL_COLUMN}' is not of type ClassLabel. Assuming 0 and 1 represent negative/positive if applicable.")

    # -----------------------------
    # SYSTEM PROMPT
    # -----------------------------
    system_prompt_labels = ""
    if LABEL_MEANING:
        system_prompt_labels = "\n".join([f"- {idx} = {meaning}" for idx, meaning in LABEL_MEANING.items()])
        SYSTEM_PROMPT = f"""
You are a classification system.

Classify the given text.

Label meanings:
{system_prompt_labels}

Rules:
- Return ONLY the number corresponding to the label
- Do NOT return words
- Do NOT explain
"""
    else:
        # Fallback prompts for specific datasets if ClassLabel info is missing or generic
        if dataset_name == "stanfordnlp/imdb" or dataset_name == "yelp_polarity" or dataset_name == "SetFit/amazon_polarity" or dataset_name == "glue/sst2":
            SYSTEM_PROMPT = """
You are a sentiment classification system.

Classify the sentiment of the given movie review.

Label meanings:
- 0 = negative sentiment
- 1 = positive sentiment

Rules:
- Return ONLY the number 0 or 1
- Do NOT return words
- Do NOT explain
"""
        elif dataset_name == "ag_news":
            SYSTEM_PROMPT = """
You are a news article classification system.

Classify the category of the given news article.

Label meanings:
- 0 = World
- 1 = Sports
- 2 = Business
- 3 = Sci/Tech

Rules:
- Return ONLY the number corresponding to the label
- Do NOT return words
- Do NOT explain
"""
        elif dataset_name == "dbpedia_14":
            SYSTEM_PROMPT = """
You are an encyclopedia article classification system.

Classify the category of the given article.

Label meanings:
- 0 = Company
- 1 = EducationalInstitution
- 2 = Artist
- 3 = Athlete
- 4 = OfficeHolder
- 5 = MeanOfTransportation
- 6 = Building
- 7 = NaturalPlace
- 8 = Village
- 9 = Animal
- 10 = Plant
- 11 = Album
- 12 = Film
- 13 = WrittenWork

Rules:
- Return ONLY the number corresponding to the label
- Do NOT return words
- Do NOT explain
"""
        elif dataset_name == "dair-ai/emotion":
            SYSTEM_PROMPT = """
You are an emotion classification system.

Classify the emotion of the given text.

Label meanings:
- 0 = sadness
- 1 = joy
- 2 = love
- 3 = anger
- 4 = fear
- 5 = surprise

Rules:
- Return ONLY the number corresponding to the label
- Do NOT return words
- Do NOT explain
"""
        elif dataset_name == "CogComp/trec":
             SYSTEM_PROMPT = """
You are a question type classification system.

Classify the fine-grained category of the given question.

Label meanings:
- 0 = abbreviation (ABBR)
- 1 = entity (ENTY)
- 2 = description (DESC)
- 3 = human (HUM)
- 4 = location (LOC)
- 5 = numeric (NUM)

Rules:
- Return ONLY the number corresponding to the label
- Do NOT return words
- Do NOT explain
"""
        else:
            # Default generic prompt if no specific prompt is found and no ClassLabel
            SYSTEM_PROMPT = """
You are a classification system.

Classify the given text.

Rules:
- Return ONLY the number corresponding to the label
- Do NOT return words
- Do NOT explain
"""


    # -----------------------------
    # ASYNC EVALUATION HELPER
    # -----------------------------
    async def _async_benchmark_evaluation():
        semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

        async def run_model_async(model_id, text):
            async with semaphore:
                try:
                    response = await client.chat.completions.create(
                        model=model_id,
                        messages=[
                            {"role": "system", "content": SYSTEM_PROMPT},
                            {"role": "user", "content": text}
                        ]
                    )
                    return response.choices[0].message.content.strip()
                except Exception as e:
                    print(f"Error running model {model_id} for {dataset_name}: {e}")
                    return "-1" # Return a distinct error value

        all_predictions = []
        tasks = []
        # Create a list of tasks for each sample and each model
        for i, item in enumerate(samples):
            text = item[TEXT_COLUMN]
            true_label = str(item[LABEL_COLUMN])
            for model_name, model_id in MODELS.items():
                tasks.append((i, model_name, run_model_async(model_id, text), true_label))

        # Execute tasks concurrently with a progress bar
        # Collect raw predictions from the tasks
        raw_preds = await tqdm.gather(*[task[2] for task in tasks], desc=f"Evaluating {dataset_name}")

        # Map raw predictions back to original sample and model
        results = {name: 0 for name in MODELS}
        task_idx = 0
        for i, item in enumerate(samples):
            true_label = str(item[LABEL_COLUMN])
            for model_name, model_id in MODELS.items():
                pred = raw_preds[task_idx]
                if pred == true_label:
                    results[model_name] += 1
                task_idx += 1
        return results

    # Run the asynchronous evaluation
    evaluation_results = asyncio.run(_async_benchmark_evaluation())

    # -----------------------------
    # RESULTS
    # -----------------------------
    print("\n==============================")
    print(f"📊 RESULTS — {dataset_name}")
    print("==============================")

    accuracies = {}
    actual_sample_size = len(samples)
    if actual_sample_size == 0:
        print(f"No samples to evaluate for {dataset_name}.")
        return {name: 0.0 for name in MODELS}

    for name in MODELS:
        acc = (evaluation_results[name] / actual_sample_size) * 100
        accuracies[name] = acc
        print(f"{name} accuracy: {acc:.2f}%")

    return accuracies

print("The benchmark_dataset function has been defined with dynamic column detection, async calls, and rate limiting.")

The benchmark_dataset function has been defined with dynamic column detection, async calls, and rate limiting.


In [ ]:
import os
import random
import asyncio
from datasets import load_dataset, ClassLabel
from openai import OpenAI
from tqdm.asyncio import tqdm

# Assuming client and MAX_CONCURRENT_REQUESTS are defined globally from previous cells
# If they were not, they would need to be re-initialized or passed as arguments.
# For this refactoring, we'll assume the global client and MODELS are available.

# client = OpenAI(api_key=os.environ["OPENAI_API_KEY"]) # This was in a previous cell
# MODELS = { "gpt-5.2": "gpt-5.2", "gpt-5-mini": "gpt-5-mini" } # This was in a previous cell

MAX_CONCURRENT_REQUESTS = 5 # Define the maximum concurrent requests, assuming it's available globally

# Redefine MODELS and DATASETS_TO_BENCHMARK for this execution context
DATASETS_TO_BENCHMARK = [
    "stanfordnlp/imdb",
    "ag_news",
    "yelp_polarity",
    "dbpedia_14",
    "dair-ai/emotion",
    "CogComp/trec",
    "SetFit/amazon_polarity",
    "glue/sst2"
]

MODELS = {
    "gpt-5.2": "gpt-5.2",
    "gpt-5-mini": "gpt-5-mini"
}

# Make benchmark_dataset an async function
async def benchmark_dataset(dataset_name, sample_size, seed):
    print(f"\n--- Benchmarking: {dataset_name} ---")

    # -----------------------------
    # LOAD DATASET
    # -----------------------------
    try:
        dataset = load_dataset(dataset_name)
    except Exception as e:
        print(f"Error loading dataset {dataset_name}: {e}")
        return {name: 0.0 for name in MODELS} # Return 0 accuracy for failed dataset

    split = "train" if "train" in dataset else list(dataset.keys())[0]
    data = list(dataset[split])

    random.seed(seed)
    samples = random.sample(data, min(sample_size, len(data)))

    # -----------------------------
    # DYNAMIC COLUMN DETECTION
    # -----------------------------
    TEXT_COLUMN = None
    LABEL_COLUMN = None

    # Common text column names
    text_col_candidates = ["text", "sentence", "review", "question", "tweet", "comment"]
    # Common label column names
    label_col_candidates = ["label", "labels", "target", "sentiment", "class"]

    features = dataset[split].features
    column_names = dataset[split].column_names

    for col in text_col_candidates:
        if col in column_names:
            TEXT_COLUMN = col
            break

    for col in label_col_candidates:
        if col in column_names:
            # Check if it's a ClassLabel type for better confidence
            if isinstance(features[col], ClassLabel):
                LABEL_COLUMN = col
                break
    # If not found by name, check by type for ClassLabel
    if not LABEL_COLUMN:
        for col_name, feature_type in features.items():
            if isinstance(feature_type, ClassLabel):
                LABEL_COLUMN = col_name
                break

    if not TEXT_COLUMN:
        print(f"Warning: Could not dynamically detect TEXT_COLUMN for {dataset_name}. Skipping dataset.")
        return {name: 0.0 for name in MODELS}
    if not LABEL_COLUMN:
        print(f"Warning: Could not dynamically detect LABEL_COLUMN for {dataset_name}. Skipping dataset.")
        return {name: 0.0 for name in MODELS}

    print(f"Detected TEXT_COLUMN: '{TEXT_COLUMN}', LABEL_COLUMN: '{LABEL_COLUMN}'")
    print(f"Using {len(samples)} samples from '{split}' split")

    # -----------------------------
    # LABEL MEANING (Dynamically create based on ClassLabel feature)
    # -----------------------------
    LABEL_MEANING = {}
    if isinstance(features[LABEL_COLUMN], ClassLabel):
        for idx, name in enumerate(features[LABEL_COLUMN].names):
            LABEL_MEANING[idx] = name
    else:
        print(f"Warning: Label column '{LABEL_COLUMN}' is not of type ClassLabel. Assuming 0 and 1 represent negative/positive if applicable.")

    # -----------------------------
    # SYSTEM PROMPT
    # -----------------------------
    system_prompt_labels = ""
    if LABEL_MEANING:
        system_prompt_labels = "\n".join([f"- {idx} = {meaning}" for idx, meaning in LABEL_MEANING.items()])
        SYSTEM_PROMPT = f"""
You are a classification system.

Classify the given text.

Label meanings:
{system_prompt_labels}

Rules:
- Return ONLY the number corresponding to the label
- Do NOT return words
- Do NOT explain
"""
    else:
        # Fallback prompts for specific datasets if ClassLabel info is missing or generic
        if dataset_name == "stanfordnlp/imdb" or dataset_name == "yelp_polarity" or dataset_name == "SetFit/amazon_polarity" or dataset_name == "glue/sst2":
            SYSTEM_PROMPT = """
You are a sentiment classification system.

Classify the sentiment of the given movie review.

Label meanings:
- 0 = negative sentiment
- 1 = positive sentiment

Rules:
- Return ONLY the number 0 or 1
- Do NOT return words
- Do NOT explain
"""
        elif dataset_name == "ag_news":
            SYSTEM_PROMPT = """
You are a news article classification system.

Classify the category of the given news article.

Label meanings:
- 0 = World
- 1 = Sports
- 2 = Business
- 3 = Sci/Tech

Rules:
- Return ONLY the number corresponding to the label
- Do NOT return words
- Do NOT explain
"""
        elif dataset_name == "dbpedia_14":
            SYSTEM_PROMPT = """
You are an encyclopedia article classification system.

Classify the category of the given article.

Label meanings:
- 0 = Company
- 1 = EducationalInstitution
- 2 = Artist
- 3 = Athlete
- 4 = OfficeHolder
- 5 = MeanOfTransportation
- 6 = Building
- 7 = NaturalPlace
- 8 = Village
- 9 = Animal
- 10 = Plant
- 11 = Album
- 12 = Film
- 13 = WrittenWork

Rules:
- Return ONLY the number corresponding to the label
- Do NOT return words
- Do NOT explain
"""
        elif dataset_name == "dair-ai/emotion":
            SYSTEM_PROMPT = """
You are an emotion classification system.

Classify the emotion of the given text.

Label meanings:
- 0 = sadness
- 1 = joy
- 2 = love
- 3 = anger
- 4 = fear
- 5 = surprise

Rules:
- Return ONLY the number corresponding to the label
- Do NOT return words
- Do NOT explain
"""
        elif dataset_name == "CogComp/trec":
             SYSTEM_PROMPT = """
You are a question type classification system.

Classify the fine-grained category of the given question.

Label meanings:
- 0 = abbreviation (ABBR)
- 1 = entity (ENTY)
- 2 = description (DESC)
- 3 = human (HUM)
- 4 = location (LOC)
- 5 = numeric (NUM)

Rules:
- Return ONLY the number corresponding to the label
- Do NOT return words
- Do NOT explain
"""
        else:
            # Default generic prompt if no specific prompt is found and no ClassLabel
            SYSTEM_PROMPT = """
You are a classification system.

Classify the given text.

Rules:
- Return ONLY the number corresponding to the label
- Do NOT return words
- Do NOT explain
"""

    # -----------------------------
    # ASYNC EVALUATION HELPER
    # -----------------------------
    async def _async_benchmark_evaluation():
        semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

        async def run_model_async(model_id, text):
            async with semaphore:
                try:
                    # Assuming `client` is an instance of `OpenAI` and available globally
                    # If client is not defined, it would need to be passed as an argument
                    # or re-initialized from `os.environ["OPENAI_API_KEY"]`
                    response = await client.chat.completions.create(
                        model=model_id,
                        messages=[
                            {"role": "system", "content": SYSTEM_PROMPT},
                            {"role": "user", "content": text}
                        ]
                    )
                    return response.choices[0].message.content.strip()
                except Exception as e:
                    print(f"Error running model {model_id} for {dataset_name}: {e}")
                    return "-1" # Return a distinct error value

        all_predictions = []
        tasks = []
        # Create a list of tasks for each sample and each model
        for i, item in enumerate(samples):
            text = item[TEXT_COLUMN]
            true_label = str(item[LABEL_COLUMN])
            for model_name, model_id in MODELS.items():
                tasks.append((i, model_name, run_model_async(model_id, text), true_label))

        # Execute tasks concurrently with a progress bar
        raw_preds = await tqdm.gather(*[task[2] for task in tasks], desc=f"Evaluating {dataset_name}")

        # Map raw predictions back to original sample and model
        results = {name: 0 for name in MODELS}
        task_idx = 0
        for i, item in enumerate(samples):
            true_label = str(item[LABEL_COLUMN])
            for model_name, model_id in MODELS.items():
                pred = raw_preds[task_idx]
                # Normalize model output and compare. Ensure prediction is valid (not '-1')
                if pred == true_label:
                    results[model_name] += 1
                task_idx += 1
        return results

    # Run the asynchronous evaluation
    evaluation_results = await _async_benchmark_evaluation() # Changed from asyncio.run to await

    # -----------------------------
    # RESULTS
    # -----------------------------
    print("\n==============================")
    print(f"📊 RESULTS — {dataset_name}")
    print("==============================")

    accuracies = {}
    actual_sample_size = len(samples)
    if actual_sample_size == 0:
        print(f"No samples to evaluate for {dataset_name}.")
        return {name: 0.0 for name in MODELS}

    for name in MODELS:
        acc = (evaluation_results[name] / actual_sample_size) * 100
        accuracies[name] = acc
        print(f"{name} accuracy: {acc:.2f}%")

    return accuracies

# Main async function to orchestrate the benchmarking process for all datasets
async def main_benchmark_orchestrator():
    global client # Ensure client is accessible if not already in global scope
    try:
        # Ensure OPENAI_API_KEY is loaded for the client
        if "OPENAI_API_KEY" not in os.environ:
            from google.colab import userdata
            os.environ["OPENAI_API_KEY"] = userdata.get('Openai_key')
            print("OpenAI API key loaded from Colab secrets.")
        client = OpenAI(api_key=os.environ["OPENAI_API_KEY"]) # Initialize client here to ensure it's available
    except Exception as e:
        print(f"Error initializing OpenAI client: {e}. Please ensure 'Openai_key' is set in Colab secrets.")
        return

    all_benchmarking_results = {}

    SAMPLE_SIZE = 500
    SEED = 42

    for dataset_name in DATASETS_TO_BENCHMARK:
        dataset_accuracies = await benchmark_dataset(dataset_name, SAMPLE_SIZE, SEED)
        all_benchmarking_results[dataset_name] = dataset_accuracies

    print("\n--- All Benchmarking Results ---")
    for dataset, results in all_benchmarking_results.items():
        print(f"\nDataset: {dataset}")
        for model, accuracy in results.items():
            print(f"  {model}: {accuracy:.2f}%")

# Run the main orchestrator once to avoid RuntimeError
# Check if an event loop is already running, if so, use the existing loop
try:
    loop = asyncio.get_running_loop()
except RuntimeError:
    loop = None

if loop and loop.is_running():
    print("An event loop is already running. Scheduling tasks...")
    # Schedule the coroutine to run on the existing event loop
    task = loop.create_task(main_benchmark_orchestrator())
    # In a notebook, it might be necessary to wait for the task, but just creating it
    # allows it to run. For sequential execution of cells, this might be tricky
    # and often requires a full restart if things get stuck.
    # For a robust solution, running the loop explicitly is better.
    # However, since `asyncio.run` is what caused the error in the first place
    # by trying to start a new loop, we must avoid it if a loop is active.
    # The best practice is to run `asyncio.run` only once at the top level.
    # If this block is executed in a context where a loop is ALREADY running (e.g. from a previous cell),
    # it must be handled.
    # A common workaround for notebooks is to use nest_asyncio.
    # For simplicity and direct fix to the error, I will assume a fresh state for `asyncio.run`.
    # Re-evaluating the strategy: The core issue is `asyncio.run` being called inside a loop/multiple times.
    # The provided fix (wrapping in one async main and calling `asyncio.run` once) is correct.
    # The `RuntimeError` implies `asyncio.run` was called on a previous cell, and then tried again.
    # So the whole block, including `asyncio.run(main_benchmark_orchestrator())` must be in a single execution scope.
    asyncio.run(main_benchmark_orchestrator())
else:
    asyncio.run(main_benchmark_orchestrator())


## Final Benchmarking Script

This script is a robust, asynchronous, and dynamically configured benchmarking pipeline for classification datasets, as provided by the user. It handles dataset loading, column detection, prompt generation, model evaluation, and result reporting efficiently.

In [1]:
# Install necessary libraries
!pip install datasets openai tqdm nest_asyncio

In [4]:
import os
import random
import asyncio
from datasets import load_dataset, ClassLabel
from openai import AsyncOpenAI
from tqdm.asyncio import tqdm
import nest_asyncio

# Apply nest_asyncio to allow nested event loops in environments like Colab
nest_asyncio.apply()

# -----------------------------
# CONFIG
# -----------------------------
SAMPLE_SIZE = 100
SEED = 42
MAX_CONCURRENT_REQUESTS = 5

DATASETS = [
    "stanfordnlp/imdb",
    "ag_news",
    "yelp_polarity",
    "dbpedia_14",
    "dair-ai/emotion",
    "CogComp/trec",
    "SetFit/amazon_polarity",
    "glue/sst2",
]

MODELS = {
    "gpt-5.2": "gpt-5.2",
    "gpt-5-mini": "gpt-5-mini",
}

# Initialize AsyncOpenAI client
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('Openai_key')
    print("OpenAI API key loaded from Colab secrets.")
except Exception:
    print("OpenAI API key not found in Colab secrets. Ensure 'Openai_key' is set or define 'OPENAI_API_KEY' environment variable.")

client = AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# -----------------------------
# HELPERS
# -----------------------------
def detect_columns(dataset, split):
    features = dataset[split].features
    cols = dataset[split].column_names

    text_candidates = ["text", "sentence", "review", "content"]
    label_candidates = ["label", "labels", "target"]

    text_col = next((c for c in text_candidates if c in cols), None)
    label_col = next((c for c in label_candidates if c in cols), None)

    if not text_col or not label_col:
        return None, None

    if not isinstance(features[label_col], ClassLabel):
        return None, None

    return text_col, label_col

def build_system_prompt(label_names):
    labels = "\n".join([f"- {i} = {name}" for i, name in enumerate(label_names)])
    return f"""
You are a text classification system.

Classify the given text.

Label meanings:
{labels}

Rules:
- Return ONLY the label number
- Do NOT return words
- Do NOT explain
""".strip()

# -----------------------------
# BENCHMARK ONE DATASET
# -----------------------------
async def benchmark_dataset(name):
    print(f"\n--- Benchmarking: {name} ---")

    try:
        ds = load_dataset(name)
    except Exception as e:
        print(f"❌ Failed to load {name}: {e}")
        return None

    split = "train" if "train" in ds else list(ds.keys())[0]
    text_col, label_col = detect_columns(ds, split)

    if not text_col or not label_col:
        print(f"⚠️ Skipping {name} (not pure classification or missing columns)")
        return None

    samples = random.sample(list(ds[split]), min(SAMPLE_SIZE, len(ds[split])))
    label_names = ds[split].features[label_col].names
    prompt = build_system_prompt(label_names)

    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    scores = {m: 0 for m in MODELS}

    async def run(model_id, text):
        async with semaphore:
            try:
                r = await client.chat.completions.create(
                    model=model_id,
                    messages=[
                        {"role": "system", "content": prompt},
                        {"role": "user", "content": text},
                    ],
                )
                return r.choices[0].message.content.strip()
            except Exception as e:
                print(f"Error during API call for {model_id} on {name}: {e}")
                return "-1" # Indicate an error

    tasks = []
    meta = []

    for item in samples:
        for m, mid in MODELS.items():
            tasks.append(run(mid, item[text_col]))
            meta.append((m, str(item[label_col])))

    preds = await tqdm.gather(*tasks, desc=f"Evaluating {name}")

    for (model, true), pred in zip(meta, preds):
        if pred == true:
            scores[model] += 1

    results = {
        model: round((scores[model] / len(samples)) * 100, 2)
        for model in MODELS
    }

    return results

# -----------------------------
# MAIN ORCHESTRATOR
# -----------------------------
async def main():
    random.seed(SEED)
    final_results = {}

    for ds in DATASETS:
        res = await benchmark_dataset(ds)
        if res:
            final_results[ds] = res

    print("\n==============================")
    print("📊 FINAL BENCHMARK RESULTS")
    print("==============================")
    for ds, res in final_results.items():
        print(f"\n{ds}")
        for model, acc in res.items():
            print(f"  {model}: {acc}%")

# Run the main orchestrator
asyncio.run(main())

OpenAI API key loaded from Colab secrets.

--- Benchmarking: stanfordnlp/imdb ---


Evaluating stanfordnlp/imdb: 100%|██████████| 200/200 [00:54<00:00,  3.69it/s]



--- Benchmarking: ag_news ---


Evaluating ag_news: 100%|██████████| 200/200 [00:58<00:00,  3.42it/s]



--- Benchmarking: yelp_polarity ---


...

train.jsonl:   0%|          | 0.00/1.75G [00:00<?, ?B/s]

test.jsonl:   0%|          | 0.00/195M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3600000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/400000 [00:00<?, ? examples/s]

⚠️ Skipping SetFit/amazon_polarity (not pure classification or missing columns)

--- Benchmarking: glue/sst2 ---
❌ Failed to load glue/sst2: Dataset 'glue/sst2' doesn't exist on the Hub or cannot be accessed.

📊 FINAL BENCHMARK RESULTS

stanfordnlp/imdb
  gpt-5.2: 96.0%
  gpt-5-mini: 94.0%

ag_news
  gpt-5.2: 85.0%
  gpt-5-mini: 80.0%

yelp_polarity
  gpt-5.2: 47.0%
  gpt-5-mini: 77.0%

dbpedia_14
  gpt-5.2: 96.0%
  gpt-5-mini: 97.0%

dair-ai/emotion
  gpt-5.2: 55.0%
  gpt-5-mini: 61.0%


In [12]:
import os
import random
import asyncio
from datasets import load_dataset, ClassLabel, Value # Import Value for more robust type checking
from openai import AsyncOpenAI
from tqdm.asyncio import tqdm
import nest_asyncio

# Apply nest_asyncio to allow nested event loops in environments like Colab
nest_asyncio.apply()

# -----------------------------
# CONFIG
# -----------------------------
SAMPLE_SIZE = 100 # Updated sample size as per instruction
SEED = 42
MAX_CONCURRENT_REQUESTS = 5

DATASETS = [
    "amazon_polarity",
    "amazon_reviews_multi/en",
    "tweet_eval/sentiment",
    "dair-ai/emotion", # Assuming 'emotion' refers to dair-ai/emotion dataset
    "banking77",
    "snli",
    "glue/mnli",
]

MODELS = {
    "gpt-5.2": "gpt-5.2",
    "gpt-5-mini": "gpt-5-mini",
}

# Initialize AsyncOpenAI client globally
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('Openai_key')
    print("OpenAI API key loaded from Colab secrets.")
except Exception:
    print("OpenAI API key not found in Colab secrets. Ensure 'Openai_key' is set or define 'OPENAI_API_KEY' environment variable.")

client = AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# -----------------------------
# HELPERS
# -----------------------------
def detect_columns(dataset, split):
    features = dataset[split].features
    cols = dataset[split].column_names

    # Specific handling for Natural Language Inference (NLI) datasets
    if all(c in cols for c in ["premise", "hypothesis", "label"]):
        if isinstance(features["label"], ClassLabel):
            # Returns text columns as a list, label column, and task type
            return ["premise", "hypothesis"], "label", "nli"

    # Common text column candidates (ordered by commonality/relevance)
    text_candidates = ["text", "sentence", "review_body", "review", "content", "tweet", "question", "document", "body", "description"]
    # Common label column candidates
    label_candidates = ["label", "labels", "target", "sentiment", "stars", "fine_label", "coarse_label"]

    text_col = next((c for c in text_candidates if c in cols), None)
    label_col = next((c for c in label_candidates if c in cols), None)

    if not text_col or not label_col:
        return None, None, None # Cannot detect both text and label columns

    # Check if label column is ClassLabel
    if isinstance(features[label_col], ClassLabel):
        return [text_col], label_col, "classification"
    # If not ClassLabel, check if it's an integer type (like 'stars' in amazon_reviews_multi)
    elif isinstance(features[label_col], Value) and features[label_col].dtype in ['int32', 'int64', 'uint8']:
        # Heuristic: if a numeric label column has a reasonable number of unique values,
        # treat it as a classification task.
        unique_labels = sorted(list(set(dataset[split][label_col])))
        if 1 < len(unique_labels) < 20: # Ensure it's a classification (more than 1, but not too many classes)
            return [text_col], label_col, "classification"

    return None, None, None # If no valid text/label pair or unsupported label type found

def build_system_prompt(label_names, task_type="classification"):
    labels = "\n".join([f"- {i} = {name}" for i, name in enumerate(label_names)])

    base_prompt = ""
    if task_type == "nli":
        base_prompt = "You are a natural language inference system.\n\nClassify the relationship between the given premise and hypothesis."
    else: # Default classification
        base_prompt = "You are a text classification system.\n\nClassify the given text."

    return f"""
{base_prompt}

Label meanings:
{labels}

Rules:
- Return ONLY the label number
- Do NOT return words
- Do NOT explain
""".strip()

# -----------------------------
# BENCHMARK ONE DATASET
# -----------------------------
async def benchmark_dataset(name):
    print(f"\n--- Benchmarking: {name} ---")

    try:
        ds = load_dataset(name)
    except Exception as e:
        print(f"❌ Failed to load {name}: {e}")
        return None

    # Determine the split to use (prioritize 'train', then 'validation', then 'test', then first available)
    split = None
    if "train" in ds: split = "train"
    elif "validation" in ds: split = "validation"
    elif "test" in ds: split = "test"
    elif len(ds.keys()) > 0: split = list(ds.keys())[0]

    if not split:
        print(f"❌ Failed to find a suitable split for {name}.")
        return None

    text_cols, label_col, task_type = detect_columns(ds, split)

    if not text_cols or not label_col:
        print(f"⚠️ Skipping {name} (not pure classification, missing columns, or invalid label type)")
        return None

    samples = random.sample(list(ds[split]), min(SAMPLE_SIZE, len(ds[split])))

    # Get label names dynamically based on the detected label feature
    label_feature = ds[split].features[label_col]
    label_names = []
    if isinstance(label_feature, ClassLabel):
        label_names = label_feature.names
    elif isinstance(label_feature, Value) and label_feature.dtype in ['int32', 'int64', 'uint8']:
        # Infer unique integer labels from sampled data and use them as names
        unique_labels = sorted(list(set(s[label_col] for s in samples)))
        if 1 < len(unique_labels) < 20: # Heuristic for classification
            label_names = [str(i) for i in unique_labels]
        else:
            print(f"⚠️ Skipping {name}: Numeric label '{label_col}' has too many unique values or too few for classification.")
            return None
    else:
        print(f"⚠️ Skipping {name}: Unsupported label type for '{label_col}' (neither ClassLabel nor numeric Value).")
        return None

    # Build the system prompt using detected label names and task type
    system_prompt_str = build_system_prompt(label_names, task_type)

    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    scores = {m: 0 for m in MODELS}

    async def run(model_id, text_input, semaphore_arg, prompt_arg): # Modified signature
        async with semaphore_arg:
            try:
                r = await client.chat.completions.create(
                    model=model_id,
                    messages=[
                        {"role": "system", "content": prompt_arg},
                        {"role": "user", "content": text_input},
                    ],
                )
                return r.choices[0].message.content.strip()
            except Exception as e:
                # Log the error but continue processing other predictions
                print(f"Error during API call for {model_id} on {name}: {e}")
                return "-1" # Indicate an error for this particular prediction

    tasks = []
    meta = []

    for item in samples:
        input_text = ""
        if task_type == "nli":
            # For NLI, concatenate premise and hypothesis
            input_text = f"Premise: {item[text_cols[0]]}\nHypothesis: {item[text_cols[1]]}"
        else: # Standard classification with a single text column
            input_text = item[text_cols[0]]

        for m, mid in MODELS.items():
            tasks.append(run(mid, input_text, semaphore, system_prompt_str)) # Modified call
            meta.append((m, str(item[label_col]))) # Ensure true label is string for comparison

    preds = await tqdm.gather(*tasks, desc=f"Evaluating {name}")

    for (model, true_label), pred in zip(meta, preds):
        # Only count correct if prediction is valid (not the "-1" error indicator) and matches
        if pred != "-1" and pred == true_label:
            scores[model] += 1

    # Calculate accuracy based on the number of samples
    results = {
        model: round((scores[model] / len(samples)) * 100, 2)
        for model in MODELS
    }

    return results

# -----------------------------
# MAIN ORCHESTRATOR
# -----------------------------
async def main():
    random.seed(SEED)
    final_results = {}

    for ds_name in DATASETS:
        res = await benchmark_dataset(ds_name)
        if res:
            final_results[ds_name] = res

    print("\n==============================")
    print("📊 FINAL BENCHMARK RESULTS")
    print("==============================")
    for ds_name, res in final_results.items():
        print(f"\n{ds_name}")
        for model, acc in res.items():
            print(f"  {model}: {acc}%")

# Run the main orchestrator
asyncio.run(main())

OpenAI API key loaded from Colab secrets.

--- Benchmarking: amazon_polarity ---


ERROR:asyncio:Task was destroyed but it is pending!
task: <Task cancelling name='Task-1214' coro=<main() done, defined at /tmp/ipython-input-2016919729.py:205>>
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1829' coro=<main() done, defined at /tmp/ipython-input-2527765110.py:205> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-2527765110.py", line 223, in <cell line: 0>
    asyncio.run(main())
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 133, in 


--- Benchmarking: amazon_reviews_multi/en ---
❌ Failed to load amazon_reviews_multi/en: Dataset 'amazon_reviews_multi/en' doesn't exist on the Hub or cannot be accessed.

--- Benchmarking: tweet_eval/sentiment ---
❌ Failed to load tweet_eval/sentiment: Dataset 'tweet_eval/sentiment' doesn't exist on the Hub or cannot be accessed.

--- Benchmarking: dair-ai/emotion ---


Evaluating dair-ai/emotion: 100%|██████████| 200/200 [01:06<00:00,  3.00it/s]



--- Benchmarking: banking77 ---


...

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/550152 [00:00<?, ? examples/s]

Evaluating snli: 100%|██████████| 200/200 [01:15<00:00,  2.66it/s]



--- Benchmarking: glue/mnli ---
❌ Failed to load glue/mnli: Dataset 'glue/mnli' doesn't exist on the Hub or cannot be accessed.

📊 FINAL BENCHMARK RESULTS

amazon_polarity
  gpt-5.2: 97.0%
  gpt-5-mini: 96.0%

dair-ai/emotion
  gpt-5.2: 61.0%
  gpt-5-mini: 61.0%

banking77
  gpt-5.2: 81.0%
  gpt-5-mini: 75.0%

snli
  gpt-5.2: 86.0%
  gpt-5-mini: 78.0%


In [14]:
import os
import random
import asyncio
from datasets import load_dataset, ClassLabel, Value # Import Value for more robust type checking
from openai import AsyncOpenAI
from tqdm.asyncio import tqdm
import nest_asyncio

# Apply nest_asyncio to allow nested event loops in environments like Colab
nest_asyncio.apply()

# -----------------------------
# CONFIG
# -----------------------------
SAMPLE_SIZE = 500 # Updated sample size as per instruction
SEED = 42
MAX_CONCURRENT_REQUESTS = 5

DATASETS = [
    "amazon_polarity",
    ("amazon_reviews_multi", "en"),
    ("tweet_eval", "sentiment"),
    "dair-ai/emotion",
    "banking77",
    "snli",
    "yahoo_answers_topics", # New dataset
]

MODELS = {
    "gpt-5.2": "gpt-5.2",
    "gpt-5-mini": "gpt-5-mini",
}

# Initialize AsyncOpenAI client globally
try:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get('Openai_key')
    print("OpenAI API key loaded from Colab secrets.")
except Exception:
    print("OpenAI API key not found in Colab secrets. Ensure 'Openai_key' is set or define 'OPENAI_API_KEY' environment variable.")

client = AsyncOpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# -----------------------------
# HELPERS
# -----------------------------
def detect_columns(dataset, split):
    features = dataset[split].features
    cols = dataset[split].column_names

    # Specific handling for yahoo_answers_topics
    if all(c in cols for c in ["question_title", "question_content", "best_answer", "topic"]) and "topic" in features and isinstance(features["topic"], ClassLabel):
        return ["question_title", "question_content", "best_answer"], "topic", "multi_text_concat"

    # Specific handling for Natural Language Inference (NLI) datasets
    if all(c in cols for c in ["premise", "hypothesis", "label"]) and "label" in features and isinstance(features["label"], ClassLabel):
        return ["premise", "hypothesis"], "label", "nli"

    # Common text column candidates (ordered by commonality/relevance)
    text_candidates = ["text", "sentence", "review_body", "review", "content", "tweet", "question", "description"]
    # Common label column candidates
    label_candidates = ["label", "labels", "target", "sentiment", "stars", "fine_label", "coarse_label", "topic"] # Added 'topic' for general detection

    text_col = next((c for c in text_candidates if c in cols), None)
    label_col = next((c for c in label_candidates if c in cols), None)

    # Added check for label_col existence in features as per instruction 5
    if not text_col or not label_col or label_col not in features:
        return None, None, None # Cannot detect both text and label columns, or label_col not in features

    # Check if label column is ClassLabel
    if isinstance(features[label_col], ClassLabel):
        return [text_col], label_col, "classification"
    # If not ClassLabel, check if it's an integer type (like 'stars' in amazon_reviews_multi)
    elif isinstance(features[label_col], Value) and features[label_col].dtype in ['int32', 'int64', 'uint8']:
        # Heuristic: if a numeric label column has a reasonable number of unique values,
        # treat it as a classification task.
        return [text_col], label_col, "classification"
    # Generic string label handling (e.g. if labels are strings directly like 'positive', 'negative')
    elif isinstance(features[label_col], Value) and features[label_col].dtype == 'string':
        return [text_col], label_col, "classification"

    return None, None, None # If no valid text/label pair or unsupported label type found

def build_system_prompt(label_names, task_type="classification"):
    labels = "\n".join([f"- {i} = {name}" for i, name in enumerate(label_names)])

    base_prompt = ""
    if task_type == "nli":
        base_prompt = "You are a natural language inference system.\n\nClassify the relationship between the given premise and hypothesis."
    elif task_type == "multi_text_concat":
        base_prompt = "You are a multi-text classification system.\n\nClassify the given concatenated text which consists of a question title, question content, and best answer."
    else: # Default classification
        base_prompt = "You are a text classification system.\n\nClassify the given text."

    return f"""
{base_prompt}

Label meanings:
{labels}

Rules:
- Return ONLY the label number
- Do NOT return words
- Do NOT explain
""".strip()

# -----------------------------
# BENCHMARK ONE DATASET
# -----------------------------
async def benchmark_dataset(dataset_info):
    name = dataset_info
    config_name = None
    if isinstance(dataset_info, tuple):
        name, config_name = dataset_info

    display_name = f"{name}{' (' + config_name + ')' if config_name else ''}"
    print(f"\n--- Benchmarking: {display_name} ---")

    try:
        if config_name:
            ds = load_dataset(name, config_name)
        else:
            ds = load_dataset(name)
    except Exception as e:
        print(f"❌ Failed to load {display_name}: {e}")
        return None

    # Determine the split to use (prioritize 'train', then 'validation', then 'test', then first available)
    split = None
    if "train" in ds: split = "train"
    elif "validation" in ds: split = "validation"
    elif "test" in ds: split = "test"
    elif len(ds.keys()) > 0: split = list(ds.keys())[0]

    if not split:
        print(f"❌ Failed to find a suitable split for {display_name}.")
        return None

    text_cols, label_col, task_type = detect_columns(ds, split)

    if not text_cols or not label_col:
        print(f"⚠️ Skipping {display_name} (not pure classification, missing columns, or invalid label type)")
        return None

    samples = random.sample(list(ds[split]), min(SAMPLE_SIZE, len(ds[split])))

    label_feature = ds[split].features[label_col]
    label_names = []
    if isinstance(label_feature, ClassLabel):
        label_names = label_feature.names
    elif isinstance(label_feature, Value) and label_feature.dtype in ['int32', 'int64', 'uint8']:
        # For numeric labels (e.g., 0, 1, 2, or 1-5 stars)
        all_labels_in_split = sorted(list(set(ds[split][label_col])))
        # Ensure that the labels are within a reasonable range for classification tasks
        # and that there are at least two unique labels.
        if 1 < len(all_labels_in_split) < 50: # Increased upper bound for more classes
            label_names = [str(l) for l in all_labels_in_split]
        else:
            print(f"⚠️ Skipping {display_name}: Numeric label '{label_col}' has too many unique values or too few for classification.")
            return None
    elif isinstance(label_feature, Value) and label_feature.dtype == 'string':
        # For string labels (e.g., 'positive', 'negative')
        all_labels_in_split = sorted(list(set(ds[split][label_col])))
        if 1 < len(all_labels_in_split) < 50: # Heuristic for string labels
            label_names = all_labels_in_split
        else:
            print(f"⚠️ Skipping {display_name}: String label '{label_col}' has too many unique values or too few for classification.")
            return None
    else:
        print(f"⚠️ Skipping {display_name}: Unsupported label type for '{label_col}'.")
        return None

    system_prompt_str = build_system_prompt(label_names, task_type)

    semaphore = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)
    scores = {m: 0 for m in MODELS}

    async def run(model_id, text_input, semaphore_arg, prompt_arg):
        async with semaphore_arg:
            try:
                r = await client.chat.completions.create(
                    model=model_id,
                    messages=[
                        {"role": "system", "content": prompt_arg},
                        {"role": "user", "content": text_input},
                    ],
                )
                return r.choices[0].message.content.strip()
            except Exception as e:
                # Log the error but continue processing other predictions
                print(f"Error during API call for {model_id} on {display_name}: {e}")
                return "-1" # Indicate an error for this particular prediction

    tasks = []
    meta = []

    for item in samples:
        input_text = ""
        if task_type == "nli":
            input_text = f"Premise: {item[text_cols[0]]}\nHypothesis: {item[text_cols[1]]}"
        elif task_type == "multi_text_concat":
            # Concatenate multiple text fields for yahoo_answers_topics
            input_text = f"Question Title: {item[text_cols[0]]}\nQuestion Content: {item[text_cols[1]]}\nBest Answer: {item[text_cols[2]]}"
        else: # Standard classification with a single text column
            input_text = item[text_cols[0]]

        for m, mid in MODELS.items():
            tasks.append(run(mid, input_text, semaphore, system_prompt_str))

            # The true label needs to be mapped to its string representation
            true_label_val = item[label_col]
            if isinstance(label_feature, ClassLabel):
                true_label_str = str(true_label_val) # ClassLabel values are usually integers mapped to names in prompt
            else: # If it's a Value type (int or string)
                true_label_str = str(true_label_val) # Directly use its string representation
            meta.append((m, true_label_str)) # Ensure true label is string for comparison


    preds = await tqdm.gather(*tasks, desc=f"Evaluating {display_name}")

    for (model, true_label), pred in zip(meta, preds):
        if pred != "-1" and pred == true_label:
            scores[model] += 1

    # Calculate accuracy based on the number of samples
    results = {
        model: round((scores[model] / len(samples)) * 100, 2)
        for model in MODELS
    }

    return results

# -----------------------------
# MAIN ORCHESTRATOR
# -----------------------------
async def main():
    random.seed(SEED)
    final_results = {}

    for ds_info in DATASETS:
        res = await benchmark_dataset(ds_info)
        if res:
            # Generate a consistent key for results dictionary
            key_name = ds_info
            if isinstance(ds_info, tuple):
                key_name = "_".join(ds_info) # e.g., "amazon_reviews_multi_en"
            final_results[key_name] = res

    print("\n==============================")
    print("📊 FINAL BENCHMARK RESULTS")
    print("==============================")
    for ds_name, res in final_results.items():
        print(f"\n{ds_name}")
        for model, acc in res.items():
            print(f"  {model}: {acc}%")

# Run the main orchestrator
asyncio.run(main())

OpenAI API key loaded from Colab secrets.

--- Benchmarking: amazon_polarity ---


Evaluating amazon_polarity:   0%|          | 0/1000 [00:00<?, ?it/s]ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-1836' coro=<main() done, defined at /tmp/ipython-input-2527765110.py:205> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-2527765110.py", line 223, in <cell line: 0>
    asyncio.run(main())
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "/usr/local/lib/python3.12/dist-packages/nest_asyncio.py", line 133, in _run_once
    handle._run()
  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run



--- Benchmarking: amazon_reviews_multi (en) ---


README.md: 0.00B [00:00, ?B/s]

amazon_reviews_multi.py: 0.00B [00:00, ?B/s]

...

yahoo_answers_topics/test-00000-of-00001(…):   0%|          | 0.00/21.9M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1400000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/60000 [00:00<?, ? examples/s]

Evaluating yahoo_answers_topics: 100%|██████████| 1000/1000 [05:11<00:00,  3.21it/s]


📊 FINAL BENCHMARK RESULTS

amazon_polarity
  gpt-5.2: 97.2%
  gpt-5-mini: 97.4%

tweet_eval_sentiment
  gpt-5.2: 70.4%
  gpt-5-mini: 68.6%

dair-ai/emotion
  gpt-5.2: 60.6%
  gpt-5-mini: 61.2%

banking77
  gpt-5.2: 78.8%
  gpt-5-mini: 77.6%

snli
  gpt-5.2: 81.0%
  gpt-5-mini: 78.4%

yahoo_answers_topics
  gpt-5.2: 74.0%
  gpt-5-mini: 74.6%
